In [70]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.python.ops.rnn_cell import BasicLSTMCell, MultiRNNCell, DropoutWrapper

# from tensorflow.models.rnn.ptb import reader
import reader

small config정보를 사용

In [64]:
## 클래스 설정
class SmallConfig(object):
    
    init_scale = 0.1  # 가중치 행렬 초기화 시 생성되는 값의 범위 지정
    learning_rate = 1.0  # gradient descent 시 사용할 변수
    max_grad_norm = 5 # 기울기가 커지는 것을 막아줌
    num_layers = 2 # 신경망 계층 수
    num_steps = 20 # 연속으로 처리할 데이터 양을 지정 
    hidden_size = 200 # 한 계층당 배치할 뉴런 수(셀의 수)
    max_epoch = 4
    max_max_epoch = 13  # 전체 학습
    keep_prob = 1.0 # 드롭아웃 하지 않을 확률
    lr_decay = 0.5
    batch_size = 20 # 미니배치 데이터의 크기 지정
    vocab_size = 10000

트레이닝과 테스트에 사용할 두개의 config 오브젝트 생성 

In [65]:
config = SmallConfig()
eval_config = SmallConfig()
eval_config.batch_size = 1
eval_config.num_steps = 1

PTB 모델을 만들어 주는 클래스 작성

In [66]:
class PTBModel(object):
    """The PTB model."""

    def __init__(self, config, is_training=False):
        self.batch_size = config.batch_size
        self.num_steps = config.num_steps
        input_size = [config.batch_size, config.num_steps]
        self.input_data = tf.placeholder(tf.int32, input_size)
        self.targets = tf.placeholder(tf.int32, input_size)

        lstm_cell = BasicLSTMCell(config.hidden_size, forget_bias=0.0, state_is_tuple=True)
        # SmallConfig에서는 드롭아웃이 적용되지 않습니다.
        if is_training and config.keep_prob < 1:
            lstm_cell = DropoutWrapper(lstm_cell, config.keep_prob)
        # 두개의 계층을 가진 신경망 구조를 만듭니다.
        cell = MultiRNNCell([lstm_cell] * config.num_layers, state_is_tuple=True)

        self.initial_state = cell.zero_state(config.batch_size, tf.float32)

        with tf.device("/cpu:0"):
            embedding_size = [config.vocab_size, config.hidden_size]
            embedding = tf.get_variable("embedding", embedding_size)
            inputs = tf.nn.embedding_lookup(embedding, self.input_data)

        # SmallConfig에서는 드롭아웃이 적용되지 않습니다.
        if is_training and config.keep_prob < 1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)

        # 각 배치마다 순서대로 데이터를 뽑아 셀에 입력합니다. 
        outputs = []
        state = self.initial_state
        with tf.variable_scope("RNN"):
            for time_step in range(config.num_steps):
                if time_step > 0: tf.get_variable_scope().reuse_variables()
                (cell_output, state) = cell(inputs[:, time_step, :], state)
                outputs.append(cell_output)

        # output의 크기를 20x20x200에서 400x200으로 변경합니다.
        output = tf.reshape(tf.concat(1, outputs), [-1, config.hidden_size])
        softmax_w_size = [config.hidden_size, config.vocab_size]
        softmax_w = tf.get_variable("softmax_w", softmax_w_size)
        softmax_b = tf.get_variable("softmax_b", [config.vocab_size])
        # logits의 크기는 400x10000이 됩니다.
        logits = tf.matmul(output, softmax_w) + softmax_b
    
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [logits],
            [tf.reshape(self.targets, [-1])],
            [tf.ones([config.batch_size * config.num_steps])])
        self.cost = tf.reduce_sum(loss) / config.batch_size
        self.final_state = state

        if not is_training:
            return

        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        # 기울기 클리핑을 수행합니다.
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars),
                                          config.max_grad_norm)
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

epoch 처리 함수

In [67]:
def run_epoch(session, m, data, is_training=False):
    """Runs the model on the given data."""
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0
    
    eval_op = m.train_op if is_training else tf.no_op()
    
    # initial_state는 2x20x200 크기의 튜플입니다.
    state_list = []
    for c, h in m.initial_state:
        state_list.extend([c.eval(), h.eval()])
    
    ptb_iter = reader.ptb_iterator(data, m.batch_size, m.num_steps)
    for step, (x, y) in enumerate(ptb_iter):
        fetch_list = [m.cost]
        # final_state 튜플에 담겨있는 상태를 꺼내어 fetch_list에 담습니다. 
        for c, h in m.final_state:
            fetch_list.extend([c, h])
        fetch_list.append(eval_op)
        
        # 이전 스텝에서 구해진 state_list가 feed_dict로 주입됩니다.
        feed_dict = {m.input_data: x, m.targets: y}
        for i in range(len(m.initial_state)):
            c, h = m.initial_state[i]
            feed_dict[c], feed_dict[h] = state_list[i*2:(i+1)*2]
        
        # fetch_list에 담긴 final_state의 결과를 state_list로 전달 받습니다.
        cost, *state_list, _ = session.run(fetch_list, feed_dict)

        costs += cost
        iters += m.num_steps

        if is_training and step % (epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                    (step * 1.0 / epoch_size, np.exp(costs / iters),
                     iters * m.batch_size / (time.time() - start_time)))

    return np.exp(costs / iters)

In [68]:
raw_data = reader.ptb_raw_data('simple-examples/data')
train_data, valid_data, test_data, _ = raw_data

train_data, valid_data, test_data는 단어를 숫자로 바꾼 리스트 입니다.

가장 많이 나온 단어 순으로 0번 부터 시작하여 10000번 까지의 번호를 가지고 있습니다.

In [71]:
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)

    # 학습과 검증, 테스트를 위한 모델을 만듭니다.
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel(config, is_training=True)
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel(config)
        mtest = PTBModel(eval_config)
        
    tf.global_variables_initializer().run()
    
    for i in range(config.max_max_epoch):
        # lr_decay는 반복속도를 조절해 주는 역할을 합니다.
        lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
        m.assign_lr(session, config.learning_rate * lr_decay)
        print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        
        perplexity = run_epoch(session, m, train_data, is_training=True)
        print("Epoch: %d Train Perplexity: %.3f" % (i + 1, perplexity))

        perplexity = run_epoch(session, mvalid, valid_data)
        print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, perplexity))

    perplexity = run_epoch(session, mtest, test_data)
    print("Test Perplexity: %.3f" % perplexity)

Epoch: 1 Learning rate: 1.000
0.004 perplexity: 8482.503 speed: 804 wps
0.104 perplexity: 867.756 speed: 854 wps
0.204 perplexity: 634.830 speed: 837 wps
0.304 perplexity: 510.601 speed: 807 wps
0.404 perplexity: 438.445 speed: 791 wps
0.504 perplexity: 391.553 speed: 750 wps
0.604 perplexity: 351.723 speed: 720 wps
0.703 perplexity: 324.605 speed: 713 wps
0.803 perplexity: 303.167 speed: 709 wps
0.903 perplexity: 283.611 speed: 705 wps
Epoch: 1 Train Perplexity: 269.011
Epoch: 1 Valid Perplexity: 177.680
Epoch: 2 Learning rate: 1.000
0.004 perplexity: 211.603 speed: 596 wps
0.104 perplexity: 150.300 speed: 672 wps
0.204 perplexity: 157.873 speed: 664 wps
0.304 perplexity: 152.793 speed: 655 wps
0.404 perplexity: 149.960 speed: 654 wps
0.504 perplexity: 147.707 speed: 658 wps
0.604 perplexity: 143.003 speed: 667 wps
0.703 perplexity: 140.892 speed: 673 wps
0.803 perplexity: 138.970 speed: 677 wps


KeyboardInterrupt: 